In [ ]:
# !pip install openai==1.78.1 numpy==2.0.2 pandas==2.2.2 seaborn==0.13.2 matplotlib==3.10.0 scikit-learn==1.6.1

## 1. 환경 설정

### 방법 1: Google Colab Secrets (상대적으로 안전)
1. Colab에서 왼쪽 사이드바의 키 아이콘을 클릭하고 새 변수 생성(Add new secret)을 누릅니다.

2. Name 란에 OPENAI_API_KEY 문자열을 입력합니다.

3. Value 란에 OpenAI에서 발급받은 API 키를 입력합니다.

4. Notebook access 버튼을 클릭하여 활성화합니다.

In [ ]:
# 방법 1: Google Colab Secrets 사용 (상대적으로 안전)

try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
    print("Colab Secrets에서 API 키를 성공적으로 불러왔습니다.")
except Exception as e:
    print("Colab Secrets를 사용할 수 없습니다.")
    print("대신 직접 입력 방법을 사용하세요.")
    api_key = None

### 방법 2: API 키를 직접 입력
이 경우 키가 노트북 파일에 저장될 수 있으므로 보안에 주의합니다.

In [ ]:
# 방법 2: 직접 입력
# 보안에 주의! 실제 프로젝트에서는 사용에 주의하세요.

import getpass
api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
print("API 키가 입력되었습니다.")

In [ ]:
# OpenAI 클라이언트 초기화
from openai import OpenAI

client = OpenAI(api_key=api_key)

# API 연결 테스트
try:
    test_response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": "안녕하세요! 거기 누구 있나요?"}],
        max_tokens=20
    )
    print("OpenAI API 연결 성공!")
    print(f"테스트 응답: {test_response.choices[0].message.content}")
except Exception as e:
    print(f"API 연결 실패: {e}")

## 2. 텍스트 완성 (Chat Completion)

In [ ]:
model = "gpt-4.1-mini"  # 모델 설정

response = client.chat.completions.create(
        model=model,
        messages=[{"role":"user", "content":"너는 누구니?"}])   # 프롬프트

print(response)

### 2.1 사용된 토큰 수 카운트

In [ ]:
print(f"사용된 모델 : {response.model}")
print(f"답변 : {response.choices[0].message.content}")
print(f"입력 토큰 수 : {response.usage.prompt_tokens}")
print(f"출력 토큰 수 : {response.usage.completion_tokens}")

In [ ]:
# API 호출 결과 중 GPT의 답변을 파싱하는 함수
def parse_answer(response):
    return response.choices[0].message.content

In [ ]:
parse_answer(response)

### 2.2 시스템 프롬프트, 유저 프롬프트와 어시스턴트 프롬프트

In [ ]:
response_santa = client.chat.completions.create(
        model=model,
        messages=[{"role":"system", "content": "당신은 산타할아버지입니다."},   # 시스템 프롬프트
                  {"role": "user", "content": "안녕하세요! 당신은 누구세요?"}]  # 유저 프롬프트
    )

print("산타 역할:", parse_answer(response_santa))

In [ ]:
conversation_response = client.chat.completions.create(
        model=model,
        messages=[{"role":"system", "content": "당신은 산타할아버지입니다. 올해 나쁜 짓을 한 아이는 선물을 받을 수 없습니다."},
                  {"role": "user", "content": "안녕하세요 산타할아버지! 제게 올해 여름에 무슨 일을 했는지 아시나요?"},
                  {"role": "assistant", "content": "화가 나서 친구와 싸웠구나. 못된 말도 했구나!"}, # 어시스턴트 프롬프트
                  {"role": "user", "content": "제가 올해 크리스마스에 선물을 받을 수 있을까요?"}]
    )

# 이와 같이 어시스턴트 프롬프트를 삽입할 경우, 모델에게 선행된 대화의 메모리를 제공하거나 답변의 보조 도구로 사용할 수 있음
print("대화 결과:", parse_answer(conversation_response))

## 3. 텍스트 생성 파라미터
1. `max_tokens`: 최대 토큰 수로, 생성할 응답의 최대 길이 제한
2. `temperature`: 응답의 무작위성 조절

In [ ]:
import time

# 파라미터 효과를 비교하기 위한 기본 프롬프트
default_prompt = "다음 단어들로 창의적인 짧은 이야기를 만들어주세요: 로봇, 꽃, 비밀"

# 기본 파라미터 설정 함수
def generate_with_params(**params):
    """파라미터를 받아서 텍스트를 생성하는 함수"""
    default_params = {
        'model': model,
        'messages': [{"role": "user", "content": default_prompt}],
        'temperature': 0,
        'max_tokens': 150
    }

    # 파라미터 업데이트
    default_params.update(params)

    response = client.chat.completions.create(**default_params)
    return response

In [ ]:
# 1. Max Tokens 비교
token_limits = [50, 200]

for max_tokens in token_limits:
    response = generate_with_params(max_tokens=max_tokens)
    result = parse_answer(response)

    print(f"\nMax Tokens: {max_tokens}")
    print(f"출력된 토큰 수: {response.usage.completion_tokens}")
    print(f"결과: {result}")
    print("-" * 50)
    time.sleep(1)

In [ ]:
# 2. Temperature 비교
temperatures = [0, 0.7, 2.0]

for temp in temperatures:
    response = generate_with_params(temperature=temp)
    result = parse_answer(response)

    print(f"\nTemperature: {temp}")
    print(f"결과: {result}")
    print("-" * 50)
    time.sleep(1)

## 4. Vision API
이미지와 텍스트를 동시에 이해할 수 있는 멀티모달 모델의 성능 파악

In [ ]:
import base64
import io

In [ ]:
# 이미지 분석 함수
def analyze_image_from_url(image_url, prompt="이 이미지를 자세히 설명해주세요."):
    """URL의 이미지를 분석합니다"""

    response = client.chat.completions.create(
        model=model,
        messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]
            }],
        max_tokens=300)
    return parse_answer(response)

def analyze_image_base64(base64_image, prompt="이 이미지를 자세히 설명해주세요."):
    """Base64 인코딩된 이미지를 분석합니다"""

    response = client.chat.completions.create(
        model=model,
        messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"}
                    }
                ]
            }],
        max_tokens=300)
    return parse_answer(response)

### 4.1. 이미지 분석
아래 이미지와 이에 대한 질문을 입력하여, 멀티모달 처리 능력을 파악
![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/1280px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [ ]:
# 예제 이미지 분석 (공개 URL 사용)
sample_image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/1280px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

print("이미지 분석 중...")
analysis = analyze_image_from_url(sample_image_url)
print("이미지 분석 결과:")
print(analysis)

# 다양한 분석 질문 예제
analysis_questions = [
    "이 이미지의 주된 색상은 무엇인가요?",
    "이 장소는 어떤 느낌을 주나요?",
    "이 이미지에서 볼 수 있는 자연 요소들을 나열해주세요."
]

print("다양한 관점에서의 이미지 분석:")

for i, question in enumerate(analysis_questions, 1):
    print(f"\n{i}. {question}")
    result = analyze_image_from_url(sample_image_url, question)
    print(f"답변: {result}")
    time.sleep(1)

### 4.2 그래프 기반 답변
모델이 차트의 형태를 파악하고, 이를 답변의 기반으로 사용할 수 있는지를 판단

In [ ]:
# 차트/그래프 이미지 생성 및 분석 예제
def create_sample_chart():
    # 데이터 생성
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
    sales = [135, 120, 108, 92, 130, 195]

    # 차트 생성
    plt.figure(figsize=(10, 6))
    plt.plot(months, sales, marker='o', linewidth=2, markersize=8)
    plt.title('Monthly sales', fontsize=16, fontweight='bold')
    plt.xlabel('Month', fontsize=12)
    plt.ylabel('Sold ($1M)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    # 이미지를 base64로 변환
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', dpi=150, bbox_inches='tight')
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
    plt.show()

    return image_base64

# 차트 생성 및 분석
print("차트 생성 중...")
chart_base64 = create_sample_chart()

# 차트 분석
chart_analysis_prompt = """
이 차트를 분석해서 다음 내용을 알려주세요:
1. 차트의 종류와 표현하는 데이터
2. 트렌드나 패턴
3. 주요 인사이트나 발견사항
"""

print("차트 분석 중...")
chart_analysis = analyze_image_base64(chart_base64, chart_analysis_prompt)
print("차트 분석 결과:")
print(chart_analysis)